# Named Entity Recognition Using BERT

### Required packages
* pytorch-pretrained-bert
* pandas
* seqeval

In [22]:
import sys
import os
import yaml
import random
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from seqeval.metrics import f1_score

import torch

from pytorch_pretrained_bert.tokenization import BertTokenizer
from torch.optim import Adam

bert_utils_path = os.path.abspath('../../utils_nlp/bert')
if bert_utils_path not in sys.path:
    sys.path.insert(0, bert_utils_path)

from configs import BERTFineTuneConfig
from bert_data_utils import KaggleNERProcessor
from bert_utils import (BertTokenClassifier, 
                        convert_examples_to_token_features, 
                        print_dict, 
                        get_device)


In [2]:
config_file = "config.yaml"

## Configurations

In [3]:
with open(config_file, 'r') as ymlfile:
    config_dict = yaml.safe_load(ymlfile)

print_dict(config_dict)

{TrainConfig: {'batch_size': 32, 'num_train_epochs': 3}
ModelConfig: {'bert_model': 'bert-base-uncased', 'max_seq_length': 75, 'do_lower_case': True}
OptimizerConfig: {'optimizer_name': 'BertAdam', 'learning_rate': 3e-05, 'no_decay_params': ['bias', 'gamma', 'beta'], 'params_weight_decay': 0.01, 'clip_gradient': True, 'max_gradient_norm': 1.0}}


In [4]:
config = BERTFineTuneConfig(config_dict)

## Preprocess Data

### Create training and validation examples
KaggleNERProcessor is a dataset specific class that generates training and evaluation examples in the format accepted by all utility functions. 

In [5]:
kaggle_ner_processor = KaggleNERProcessor(data_dir="./data/NER/ner_dataset.csv", dev_percentage = 0.1)

In [6]:
train_examples = kaggle_ner_processor.get_train_examples(data_dir="./data/NER/ner_dataset.csv")
dev_examples = kaggle_ner_processor.get_dev_examples(data_dir="./data/NER/ner_dataset.csv")
label_list = kaggle_ner_processor.get_labels()

In [7]:
print('Sample sentence: \n{}\n'.format(train_examples[0].text_a))
print('Sample sentence labels: \n{}\n'.format(train_examples[1].label))

Sample sentence: 
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .

Sample sentence labels: 
['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O']



### Convert examples to features
The function `convert_examples_to_token_features` converts raw string data to numerical features, involving the following steps:
1. Tokenization
2. Convert tokens and labels to numerical values
3. Sequence padding or truncation

In [8]:
label_map = {label: i for i, label in enumerate(label_list)}

In [9]:
tokenizer = BertTokenizer.from_pretrained(config.bert_model,
                                          do_lower_case=config.do_lower_case)

train_features = convert_examples_to_token_features(examples=train_examples,
                                                    tokenizer=tokenizer,
                                                    label_map=label_map, 
                                                    config=config)
dev_features = convert_examples_to_token_features(examples=dev_examples,
                                                  tokenizer=tokenizer,
                                                  label_map=label_map, 
                                                  config=config)

In [10]:
print("Sample token id:\n{}\n".format(train_features[0].input_ids))
print("Sample attention mask:\n{}\n".format(train_features[0].input_mask))
print("Sample label ids:\n{}\n".format(train_features[0].label_id))

Sample token id:
[5190, 1997, 28337, 2031, 9847, 2083, 2414, 2000, 6186, 1996, 2162, 1999, 5712, 1998, 5157, 1996, 10534, 1997, 2329, 3629, 2013, 2008, 2406, 1012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Sample attention mask:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Sample label ids:
[6, 6, 6, 6, 6, 6, 9, 6, 6, 6, 6, 6, 9, 6, 6, 6, 6, 6, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,

## Create Token Classifier

In [11]:
device, n_gpu = get_device()
token_classifier = BertTokenClassifier(config=config, 
                                       label_map=label_map, 
                                       device=device, 
                                       n_gpu=n_gpu)

BERT fine tune configurations:
batch_size=32
num_train_epochs=3
bert_model=bert-base-uncased
max_seq_length=75
do_lower_case=True
optimizer_name=BertAdam
learning_rate=3e-05
no_decay_params=['bias', 'gamma', 'beta']
params_weight_decay=0.01
clip_gradient=True
max_gradient_norm=1.0


t_total value of -1 results in schedule not being applied


## Train Model

In [12]:
token_classifier.fit(train_features)

Iteration:   9%|▉         | 126/1349 [02:06<20:12,  1.01it/s]


Iteration:  19%|█▊        | 252/1349 [04:12<18:10,  1.01it/s]


Iteration:  28%|██▊       | 378/1349 [06:18<16:35,  1.03s/it]


Iteration:  37%|███▋      | 504/1349 [08:26<14:36,  1.04s/it]


Iteration:  47%|████▋     | 630/1349 [10:33<12:00,  1.00s/it]


...

Iteration:  75%|███████▍  | 1008/1349 [17:00<05:44,  1.01s/it]


Iteration:  84%|████████▍ | 1132/1349 [19:06<03:39,  1.01s/it]


Iteration:  93%|█████████▎| 1256/1349 [21:12<01:34,  1.01s/it]


Epoch: 100%|██████████| 3/3 [1:08:14<00:00, 1364.10s/it]3it/s]

Train loss: 0.057512551753210794


## Evaluate Model

In [20]:
pred_tags, true_tags = token_classifier.predict(dev_features)

Validation loss: 0.09586639727155367


In [24]:
f1_score(pred_tags, true_tags)

0.4521074735617062